In [ ]:
import gensim
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import spacy
import string
import tensorflow as tf

In [ ]:
gensim.__version__

In [ ]:
np.random.seed(42)

In [ ]:
import gensim.downloader as api
print(list(api.info()['models'].keys()))

In [ ]:
wv=api.load('word2vec-google-news-300')

In [ ]:
wv.save('vectors.kv')

In [ ]:
len(wv['apple'])

In [ ]:
wv.similarity("phone", "macbook")

In [ ]:
wv.most_similar(positive=['phone','iphone'], topn=5)

In [ ]:
data=pd.read_csv('one_output_dataset.csv')

In [ ]:
data

In [ ]:
def sentence_vec(sentence):
    vector_size=wv.vector_size
    wv_res=np.zeros(vector_size)
    ctr=1
    for w in sentence:
        if w in wv:
            ctr+=1
            wv_res+=wv[w]
        wv_res=wv_res/ctr
        return wv_res

In [ ]:
def spacy_tokenizer(sentence):
    #creating token object, which is used to create documents with linguistic annotations
    doc=nlp(sentence)
    #lemmatizing each token and lowercase conversion
    mytokens=[word.lemma_.lower().strip() for word in doc]
    # removing stop words
    mytokens=[word for word in mytokens if word not in stop_words and word not in punctuations]
    return mytokens

In [ ]:
nlp=spacy.load("en_core_web_sm")
stop_words=nlp.Defaults.stop_words
punctuations=string.punctuation

In [ ]:
data['tokens']=data['speech'].apply(spacy_tokenizer)

In [ ]:
data.head()

In [ ]:
len(data['tokens'])

In [ ]:
data['vec']=data['tokens'].apply(sentence_vec)

In [ ]:
data.head()

In [ ]:
speech_vectors=data['vec'].values
labels=data['emotion'].values


In [ ]:
df=data

In [ ]:
df.pop('speech')
df.pop('tokens')

In [ ]:
columns_titles = ["vec","emotion"]
df=df.reindex(columns=columns_titles)
df

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(speech_vectors,labels,test_size=0.2,random_state=7)

In [ ]:
X_train=X_train.tolist()
X_test=X_test.tolist()
y_train=y_train.tolist()
y_test=y_test.tolist()

In [ ]:
X_train=tf.convert_to_tensor(X_train)
X_test=tf.convert_to_tensor(X_test)
y_train=tf.convert_to_tensor(y_train)
y_test=tf.convert_to_tensor(y_test)

In [ ]:
X_train

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(10000).batch(2)

In [ ]:
type(train_dataset)

In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_dataset = test_dataset.shuffle(10000).batch(2)

In [ ]:
from transformers import AlbertTokenizer, TFAlbertForSequenceClassification, AdamW
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

In [ ]:
model = TFAlbertForSequenceClassification.from_pretrained('albert-large-v2', num_labels=6)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']

In [ ]:
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
model.summary()

In [ ]:
model.fit(train_dataset, epochs=3, validation_data=test_dataset)